In [7]:
# Importing the Required Libraries
import torch
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
import torch.nn as nn
import torch.optim as optim

In [9]:
# Loading a pre-trained model optimized for sentence embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # 'all-MiniLM-L6-v2' is a Lightweight BERT-based model optimized for sentence embeddings that is well-suited for similar tasks.


# Example sentences for testing
sentences = ["Enter your coupon code at checkout to unlock exclusive discounts.",
             "Use promo codes at checkout to enjoy extra savings on your favorite products.",
             "Leave a review and share your experience with others."]

# Define some custom categories for these sentences
categories = {
    "Promotion": 0,  # Label for promotion-related sentences
    "Customer Feedback": 1,  # Label for customer feedback sentences
}

# Corresponding labels based on the custom categories
labels = [categories["Promotion"], categories["Promotion"], categories["Customer Feedback"]]

# Generate embeddings for the sentences
embeddings = model.encode(sentences)

# Print shape of the batch embeddings
print("Embedding Shape:", embeddings.shape)

Embedding Shape: (3, 384)


We have 3 sentences in our batch each represented by a 384-dimensional vector.

In [10]:
# Example sentences for testing
sentences = ["Your coupon code exclusive discounts is expired.",
             "Use promo codes at checkout is not mandatory.",
             "Your review is appreciated."]

# Generate embeddings for the sentences
embeddings = model.encode(sentences, convert_to_tensor=True)

# Define the classification and sentiment heads (no class structure)
classification_head = nn.Linear(384, 4)  # 384-dimensional input, 4 classes for sentence classification
sentiment_head = nn.Linear(384, 3)  # 384-dimensional input, 3 classes for sentiment analysis

# Apply Softmax to the outputs to get probabilities
softmax = nn.Softmax(dim=1)

# Get the classification and sentiment outputs
classification_output = classification_head(embeddings)
sentiment_output = sentiment_head(embeddings)

# Apply Softmax to the outputs
classification_probs = softmax(classification_output)
sentiment_probs = softmax(sentiment_output)

# Map the index of max probability to category
classification_labels = ['Promotion Expiry', 'Coupon Usage', 'Customer Feedback', 'Discount Offers']
sentiment_labels = ['Negative', 'Neutral', 'Positive']

for i, sentence in enumerate(sentences):
    # Classification
    classification_index = torch.argmax(classification_probs[i]).item()
    classification_result = classification_labels[classification_index]

    # Sentiment
    sentiment_index = torch.argmax(sentiment_probs[i]).item()
    sentiment_result = sentiment_labels[sentiment_index]

    # Print out classification probabilities for debugging
    print(f"Sentence: {sentence}")
    print(f"Classification Output (Category): {classification_result}")
    print(f"Classification Probabilities: {classification_probs[i]}")
    print(f"Sentiment Output: {sentiment_result}")
    print(f"Sentiment Probabilities: {sentiment_probs[i]}")
    print("\n")

Sentence: Your coupon code exclusive discounts is expired.
Classification Output (Category): Promotion Expiry
Classification Probabilities: tensor([0.2576, 0.2510, 0.2388, 0.2525], grad_fn=<SelectBackward0>)
Sentiment Output: Positive
Sentiment Probabilities: tensor([0.3346, 0.3276, 0.3378], grad_fn=<SelectBackward0>)


Sentence: Use promo codes at checkout is not mandatory.
Classification Output (Category): Coupon Usage
Classification Probabilities: tensor([0.2543, 0.2551, 0.2367, 0.2540], grad_fn=<SelectBackward0>)
Sentiment Output: Positive
Sentiment Probabilities: tensor([0.3211, 0.3350, 0.3440], grad_fn=<SelectBackward0>)


Sentence: Your review is appreciated.
Classification Output (Category): Coupon Usage
Classification Probabilities: tensor([0.2574, 0.2646, 0.2392, 0.2388], grad_fn=<SelectBackward0>)
Sentiment Output: Neutral
Sentiment Probabilities: tensor([0.3270, 0.3511, 0.3219], grad_fn=<SelectBackward0>)




#### Should the Entire Network be Frozen
* Freezing the entire network is typically not ideal when working with task-specific heads that need to adapt to specific problems, such as sentiment analysis or sentence classification. However, it might be useful in cases where you’re limited by computational resources and want a quick solution.

#### Should Only the Transformer Backbone be Frozen?
* This approach allows the model to learn task-specific features while retaining the powerful language representations learned during pre-training.
But if the transformer backbone is frozen, the model might not adapt perfectly to the nuances of the tasks.

* Freezing one task-specific head means the model will adapt one task while keeping the other task’s head fixed. This can be useful in a multi-task learning scenario where one task is more complex or requires more adjustment.

### Transfer learning process
* In general, transfer learning is beneficial when you have a pre-trained model that has learned useful knowledge from a large dataset. By freezing the backbone and fine-tuning the heads, you can leverage the general language knowledge and apply it to specific tasks.


In [11]:
# Training Loop

# Loss functions and optimizer
criterion_classification = nn.CrossEntropyLoss()
criterion_sentiment = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(classification_head.parameters()) + list(sentiment_head.parameters()), lr=0.001)

# Hypothetical Dataset (Sentences and Random Labels)
sentences = ["Enter your coupon code at checkout to unlock exclusive discounts.",
             "Use promo codes at checkout to enjoy extra savings on your favorite products.",
             "Leave a review and share your experience with others."]

# Simulate sentence embeddings (Replace with real embeddings in actual training)
embeddings = torch.randn(len(sentences), 384)

# Random labels for classification and sentiment tasks
classification_labels = torch.randint(0, 4, (len(sentences),))
sentiment_labels = torch.randint(0, 3, (len(sentences),))

# Training Loop (Single Epoch for Demonstration)
num_epochs = 5
for epoch in range(num_epochs):
    classification_head.train()
    sentiment_head.train()

    optimizer.zero_grad()  # Reset gradients

    # Forward pass through both heads
    classification_output = classification_head(embeddings)
    sentiment_output = sentiment_head(embeddings)

    # Compute Losses
    loss_classification = criterion_classification(classification_output, classification_labels)
    loss_sentiment = criterion_sentiment(sentiment_output, sentiment_labels)

    total_loss = loss_classification + loss_sentiment  # Combined loss for multi-task learning

    # Backward pass
    total_loss.backward()
    optimizer.step()

    # Print progress
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss.item():.4f}")

Epoch [1/5], Loss: 2.6939
Epoch [2/5], Loss: 2.2082
Epoch [3/5], Loss: 1.7780
Epoch [4/5], Loss: 1.4071
Epoch [5/5], Loss: 1.0969


  Loss is decreasing steadily across epochs, which is a good sign! It means the model is learning and adjusting its weights effectively.